In [8]:
!conda install pytorch torchvision torchaudio cudatoolkit=11.3 -c pytorch --yes
!pip install transformers
!pip install feedparser==5.2.1 pygooglenews==0.1.2 newspaper3k==0.2.8
!pip install anvil-uplink
!pip install tensorflow

In [ ]:
import anvil.server
from datetime import datetime
from newspaper import Article
from pygooglenews import GoogleNews
import string

In [ ]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline

qna = pipeline('question-answering', model='deepset/roberta-base-squad2', tokenizer='deepset/roberta-base-squad2')

In [2]:
anvil.server.connect('6US6FXVLUHXQMQTBHNZ5XZ3V-TUNB7K7SPLS2NVGN')

Connecting to wss://anvil.works/uplink
Anvil websocket open
Connected to "Default environment" as SERVER


In [22]:
def gather_news(topic, when="1d", num=10):
    gn = GoogleNews()
    news = gn.search(topic, when=when)
    all_text = []
    article_num=0
    for i in range(num):
        try:
            url = news['entries'][i]["links"][0]['href']
            article = Article(url)
            article.download()
            article.parse()
            all_text.append(article.text)
            article_num+=1
        except:
            pass
    print(f"Number of articles referenced: {article_num}")
    all_text = ",".join(all_text)
    
    return all_text


@anvil.server.callable
def ask_qna(topic, question, when="1d", num=10, top_k=10):
    print(datetime.now())
    print(f"Topic: {topic} | Question: {question}")
    qna_set = {
                    'question': question,
                    'context': gather_news(topic, when=when, num=num)
                   }
    results = qna(qna_set, top_k=top_k)
    
    print()
    
    if top_k == 1:
        results = [results]

    i=0
    for item in results:
        i+=1
        query = (topic+" "+item['answer']).translate(str.maketrans('','',string.punctuation)).replace(" ", "+")
        item.update( {"numbering":i, "url": f"https://news.google.com/search?for={query}+when:{when}&hl=en"})

    for result in results:
        print(f"Rank: {result['numbering']}\nAnswer: {result['answer']}\nLikelihood: {round(result['score'],4)}\n")

    return results

anvil.server.wait_forever()

2022-02-24 14:05:17.215056
Topic: Bitcoin | Question: When is a good time to buy Bitcoin?
Number of articles referenced: 1

Rank: 1
Answer: weeks or a couple of months
Likelihood: 0.1353

Rank: 2
Answer: “weeks or a couple of months
Likelihood: 0.0624

Rank: 3
Answer: November
Likelihood: 0.0384

2022-02-24 14:05:50.170749
Topic: lg energy solution | Question: What is happening?
Number of articles referenced: 1

Rank: 1
Answer: digitally transform its entire global facilities into data-centric operations
Likelihood: 0.1355

Rank: 2
Answer: the company continues its expansion
Likelihood: 0.0866

Rank: 3
Answer: company continues its expansion
Likelihood: 0.0749

2022-02-24 14:07:05.957929
Topic: Korean Presidential election | Question: Who is most likely to become the president?
Number of articles referenced: 1

Rank: 1
Answer: Donald Trump
Likelihood: 0.3478

Rank: 2
Answer: Yoon Suk Yeol
Likelihood: 0.049

Rank: 3
Answer: Lee Jae-myung
Likelihood: 0.0131

2022-02-24 14:07:21.439491
To